In [61]:
import os
import math
from tqdm import tqdm
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from torchvision import transforms as T
from torchmetrics.image import StructuralSimilarityIndexMeasure
import torch.utils.dlpack
import cupy as cp
from cuml import UMAP
from cuml.cluster import KMeans, HDBSCAN
from cuml.decomposition import IncrementalPCA
from cuml.manifold import TSNE
from cuml.metrics.cluster import silhouette_score

from model.models import *
from dataset.dataset import (
    device_checker,
    load_original_datasets,
    denormalize_tensor,
    denorm_mean_tensor,
    denorm_std_tensor,
    create_background_mask_otsu
)

ModuleNotFoundError: No module named 'cupy'

In [23]:
#root_dir = '/projects/dsci410_510/gz_hubble'
root_dir = '../data/gz_hubble'
weights_path = '../model/saved_models/CustomAutoencoder.pt'

# Initialize
device = device_checker()
dataset_dir = '../data/gz_hubble'  # UPDATE THIS
full_catalog, _, og_test_catalog = load_original_datasets(dataset_dir, return_og_catalogs=True)


Using device: mps

Loading original datasets from file...
Saved the new combined catalog to: ../data/gz_hubble/full_catalog.parquet


'# Create dataloaders\nsingle_view_transform, double_view_transform = create_transforms()\n_, _, test_loader, _, _, test_dataset = get_data_loaders(\n    full_catalog,\n    double_view_transform,\n    num_workers=0,\n    prefetch_factor=0,\n)'

In [29]:
images_dir = os.path.join(root_dir, "images")
image_paths = og_test_catalog['file_loc'].tolist()
total_images = len(image_paths)

LATENT_DIM = 32  # Match up with value used in model training
IMAGE_LOAD_BATCH_SIZE = 256

# Load model and weights
unet_model = UNetAutoencoder()
custom_model = CustomAutoencoder(
    activation_type='prelu',
    latent_dim=32
)
model = custom_model  # SET MODEL HERE
model.to(device)
model.load_state_dict(torch.load(weights_path, map_location=device, weights_only=True))
model.eval()

# Define transforms
mean = [0.0441, 0.0464, 0.0484]
std = [0.0712, 0.0726, 0.0713]
transform = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor(),
    T.Normalize(mean=mean, std=std)
])

# Initialize SSIM score metric
ssim_metric = StructuralSimilarityIndexMeasure(data_range=1.0).to(device)
ssim_metric.reset()
ssim_scores = []

# Pre-allocate a single large tensor to hold all latent vectors
all_latents = torch.empty(total_images, LATENT_DIM, device=device)
start_idx = 0

with torch.no_grad():
    for i in tqdm(range(0, total_images, IMAGE_LOAD_BATCH_SIZE)):
        batch_files = image_paths[i: i + IMAGE_LOAD_BATCH_SIZE]
        batch_tensors = []

        for f in batch_files:
            with Image.open(f) as pil_img:
                pil_img = pil_img.convert("RGB")
                t_img = transform(pil_img)
            batch_tensors.append(t_img)

        batch_input = torch.stack(batch_tensors, dim=0).to(device)

        # Obtain reconstruction and latent vectors
        recon, latents = model(batch_input, return_latent=True)

        # Store latent representations
        end_idx = start_idx + latents.shape[0]
        all_latents[start_idx:end_idx] = latents
        start_idx = end_idx

        # Denormalize inputs and reconstructions
        batch_input_denorm = denormalize_tensor(batch_input)
        recon_denorm = denormalize_tensor(recon)

        # Create foreground/galaxy masks for each image in batch
        galaxy_masks = []
        for img in batch_input_denorm:
            background_mask = create_background_mask_otsu(img)
            galaxy_mask = 1 - background_mask
            galaxy_masks.append(galaxy_mask)
        galaxy_mask_tensor = torch.stack(galaxy_masks, dim=0).unsqueeze(1).to(device)

        # Apply galaxy mask to both target and reconstruction
        input_galaxy = batch_input_denorm * galaxy_mask_tensor
        recon_galaxy = recon_denorm * galaxy_mask_tensor

        # Compute SSIM only for galaxy region
        batch_ssim = ssim_metric(recon_galaxy, input_galaxy)
        ssim_scores.append(batch_ssim.item())


# Compute overall average SSIM across all batches
avg_ssim = sum(ssim_scores) / len(ssim_scores)
print("Average SSIM on test set galaxy regions:", np.round(avg_ssim, 3))

# Zero-copy conversion via DLPack
latents_dlpack = torch.utils.dlpack.to_dlpack(all_latents)
all_latents_cp = cp.from_dlpack(latents_dlpack)
print("Cupy array shape:", all_latents_cp.shape)

100%|██████████| 32/32 [00:42<00:00,  1.33s/it]

Latents shape: torch.Size([2000, 32])
Average SSIM on test set galaxy regions: 0.936
NumPy array shape: (2000, 32)


In [31]:
# -----
# PCA
# -----

pca_components = 16
batch_size = 256

ipca = IncrementalPCA(n_components=pca_components, batch_size=batch_size, verbose=True)

num_samples = all_latents_cp.shape[0]
i = 0
while i < num_samples:
    end = min(i + batch_size, num_samples)
    current_batch = all_latents_cp[i:end]

    # Merge if the final batch is smaller than the PCA dimension
    if (end - i) < pca_components and i > 0:
        prev_start = i - batch_size
        merged_batch = np.concatenate([all_latents_cp[prev_start:i], current_batch], axis=0)
        batch_gpu = cp.asarray(merged_batch, dtype=cp.float32)
        ipca.partial_fit(batch_gpu)
    else:
        batch_gpu = cp.asarray(current_batch, dtype=cp.float32)
        ipca.partial_fit(batch_gpu)

    i += batch_size

# Transform the entire dataset into PCA space on GPU
data_pca_gpu = cp.asarray(all_latents_cp, dtype=cp.float32)
all_latents = ipca.transform(data_pca_gpu)

NameError: name 'cp' is not defined

In [ ]:
# Convert latent vectors to GPU memory directly
all_latents = cp.asarray(all_latents_cp, dtype=cp.float32)

# -------------------------
# HDBSCAN Clustering
# -------------------------
hdbscan_model = HDBSCAN(
    min_cluster_size=25
)
cluster_labels = hdbscan_model.fit_predict(all_latents)

unique_labels_cp, counts_cp = cp.unique(cluster_labels, return_counts=True)
unique_labels = unique_labels_cp.get()
counts = counts_cp.get()

num_clusters = (unique_labels != -1).sum()
outlier_count = counts[unique_labels == -1].sum() if -1 in unique_labels else 0
outlier_fraction = outlier_count / cluster_labels.size

print(f"Number of clusters (excluding outliers): {num_clusters}")
print(f"Outliers count: {outlier_count} ({outlier_fraction:.2%} of all points)")

print("\nHDBSCAN cluster label -> count")
for label, count in zip(unique_labels, counts):
    print(f"{label:>3} -> {count}")

# Compute silhouette score on subset (excluding outliers)
sample_frac = 0.1
num_samples_subset = min(int(sample_frac * all_latents.shape[0]), 10000)
indices = cp.random.permutation(all_latents.shape[0])[:num_samples_subset]

data_sample = all_latents[indices]
labels_sample = cluster_labels[indices]
mask = (labels_sample != -1)

hdbscan_sil_score = silhouette_score(data_sample[mask], labels_sample[mask])
print(f"\nHDBSCAN Silhouette score: {hdbscan_sil_score:.3f}")

# -------------------------
# K-means Clustering
# -------------------------

num_clusters = 6
kmeans_model = KMeans(num_clusters=num_clusters, n_init="auto")
kmeans_model.fit(all_latents)

# Retrieve cluster labels
kmeans_labels = kmeans_model.labels_

# Check cluster counts
unique_kmeans_labels_cp, kmeans_counts_cp = cp.unique(kmeans_labels, return_counts=True)
unique_kmeans_labels = unique_kmeans_labels_cp.get()
kmeans_counts = kmeans_counts_cp.get()

print(f"\nKMeans with n_clusters={num_clusters}")
for label, count in zip(unique_kmeans_labels, kmeans_counts):
    print(f"Cluster {label} -> {count} points")

# Compute silhouette score
kmeans_sil_score = silhouette_score(all_latents, kmeans_labels)
print(f"\nKMeans Silhouette score: {kmeans_sil_score:.3f}")

In [ ]:
# ---------------------------
# Cluster Plotting Function
# ---------------------------

def plot_clusters(data_2d, labels, title="", alpha_outliers=0.1, s_outliers=2, alpha_clusters=0.5, s_clusters=10):
    # Convert to NumPy (if needed)
    if isinstance(data_2d, cp.ndarray):
        data_2d = cp.asnumpy(data_2d)
    if isinstance(labels, cp.ndarray):
        labels = cp.asnumpy(labels)

    unique_labels = np.unique(labels)
    num_clusters = np.unique(labels[labels != -1]).size

    # Build a colormap for the clusters
    colors = plt.cm.viridis_r(np.linspace(0, 1, num_clusters))
    label_color_map = {lb: colors[i] for i, lb in
                       enumerate(sorted(unique_labels[unique_labels != -1]))}
    label_color_map[-1] = (0, 0, 0, 1)  # Outliers set to black

    plt.figure(figsize=(8, 6))

    for lb in unique_labels:
        mask = (labels == lb)

        # Use smaller size and lower alpha for outliers
        plot_alpha = alpha_outliers if lb == -1 else alpha_clusters
        plot_size = s_outliers if lb == -1 else s_clusters

        plt.scatter(
            data_2d[mask, 0],
            data_2d[mask, 1],
            c=[label_color_map[lb]],
            label=f"Cluster {lb}" if lb != -1 else "Outliers",
            s=plot_size,
            alpha=plot_alpha
        )

    plt.title(f"{title}  (Clusters: {num_clusters})")
    plt.xlabel("Component 1")
    plt.ylabel("Component 2")
    plt.legend(markerscale=2)
    plt.show()


# ------------------------------------
#  t-SNE Visualization of Clusters
# ------------------------------------

cluster_labels_hdbscan = hdbscan_model.labels_
cluster_labels_kmeans = kmeans_labels

"""tsne = TSNE(
    n_components=2,
    perplexity=50,
    n_neighbors=250,
    learning_rate=20,
    n_iter=3000,
    verbose=False
)"""

tsne = TSNE(
    n_components=2,
    n_iter=3000,
    verbose=False
)
data_2d_tsne = tsne.fit_transform(all_latents)

#plot_clusters(data_2d_tsne, cluster_labels_hdbscan, title="t-SNE Visualization (HDBSCAN)")
plot_clusters(data_2d_tsne, cluster_labels_kmeans, title="t-SNE Visualization (K-means)")

# ----------------------------------
#  UMAP Visualization of Clusters
# ----------------------------------

"""umap_2d = UMAP(
    n_components=2,
    n_neighbors=30,
    learning_rate=0.5,
    n_epochs=3000,
    verbose=False
)"""

umap_2d = UMAP(
    n_components=2,
    n_neighbors=30,
    n_epochs=3000,
    verbose=False
)
data_2d_umap = umap_2d.fit_transform(all_latents)

#plot_clusters(data_2d_umap, cluster_labels_hdbscan, title="UMAP Visualization (HDBSCAN)")
plot_clusters(data_2d_umap, cluster_labels_kmeans, title="UMAP Visualization (K-means)")

In [ ]:
# -------------------------------------
#  3-D UMAP Visualization of Clusters
# -------------------------------------

"""umap_3d = UMAP(
    n_components=3,
    n_neighbors=30,
    learning_rate=0.5,
    n_epochs=3_000,
    verbose=False
    )"""

umap_3d = UMAP(
    n_components=3,
    verbose=False
    )

latent_3d_gpu = umap_3d.fit_transform(all_latents)

# Send to CPU for plotting
data_3d = cp.asnumpy(latent_3d_gpu)
cluster_labels_np = cp.asnumpy(cluster_labels)

# Create plot
fig = go.Figure()
fig.add_trace(go.Scatter3d(
    x=data_3d[:, 0],
    y=data_3d[:, 1],
    z=data_3d[:, 2],
    mode='markers',
    marker=dict(
        size=4,
        color=cluster_labels_np,
        colorscale='Viridis',
        opacity=0.25
    ),
    name='Data points'
))

fig.update_layout(
    title="Interactive 3-D UMAP Visualization",
    scene=dict(
        xaxis_title='Component 1',
        yaxis_title='Component 2',
        zaxis_title='Component 3'
    )
)

fig.show()

In [ ]:
# -----------------------------------------------
# Function for Visualizing Cluster Example Images
# -------------------------------------------------

def plot_cluster_samples(catalog, cluster_labels, cluster_id, n=9, image_col="file_loc"):
    # Grab indices
    idxs = [i for i, lbl in enumerate(cluster_labels) if lbl == cluster_id]
    if not idxs:
        print(f"No samples found for cluster {cluster_id}.")
        return

    # Convert indices into a catalog of just that cluster
    cluster_catalog = catalog.iloc[idxs].copy()

    # Randomly sample up to n images
    if len(cluster_catalog) > n:
        cluster_catalog = cluster_catalog.sample(n)
    else:
        cluster_catalog = cluster_catalog.sample(frac=1.0)

    # Initialize layout
    num_samples = len(cluster_catalog)
    rows = int(math.ceil(num_samples ** 0.5))
    cols = int(math.ceil(num_samples / rows))
    fig, axes = plt.subplots(rows, cols, figsize=(2.5 * cols, 2.5 * rows))
    if rows * cols == 1:
        axes = [axes]
    else:
        axes = axes.flatten()

    # Plot each sample
    for ax, (_, row) in zip(axes, cluster_catalog.iterrows()):
        file_path = row[image_col]
        try:
            with Image.open(file_path) as img:
                ax.imshow(img)
            ax.set_title(f"Cluster {cluster_id}", fontsize=9)
        except Exception as e:
            print(f"Error loading '{file_path}': {e}")
        ax.axis("off")
    for leftover_ax in axes[num_samples:]:
        leftover_ax.axis("off")

    plt.tight_layout()
    plt.show()


# -------------------------------
# Visualizing Cluster Examples
# -------------------------------

for cluster_id in range(7):
    print(f"\nShowing samples from cluster {cluster_id}...")
    plot_cluster_samples(
        catalog=og_test_catalog,
        cluster_labels=cluster_labels_hdbscan,
        cluster_id=cluster_id,
        n=9,
        image_col="file_loc"
    )